In [61]:
import numpy as np
import pandas as pd
import os  

In [175]:
clientes = pd.read_csv('data\e-comerce_Olist_dataset\olist_customers_dataset.csv')
geolocalizacion = pd.read_csv('data\e-comerce_Olist_dataset\olist_geolocation_dataset_coregido.csv')


items = pd.read_csv('data\e-comerce_Olist_dataset\olist_order_items_dataset.csv', parse_dates=['shipping_limit_date'])
metodo_de_pago = pd.read_csv('data\e-comerce_Olist_dataset\olist_order_payments_dataset.csv')
reviews = pd.read_csv('data\e-comerce_Olist_dataset\olist_order_reviews_dataset.csv', parse_dates=['review_creation_date', 'review_answer_timestamp'])
ordenes = pd.read_csv('data\e-comerce_Olist_dataset\olist_orders_dataset.csv', parse_dates=['order_purchase_timestamp', 'order_approved_at','order_delivered_carrier_date','order_delivered_customer_date','order_estimated_delivery_date'])
productos = pd.read_csv('data\e-comerce_Olist_dataset\olist_products_dataset.csv')
vendedores = pd.read_csv('data\e-comerce_Olist_dataset\olist_sellers_dataset.csv')
productos_info = pd.read_csv('data\e-comerce_Olist_dataset\product_category_name_translation.csv')

marketing_ofertas_cerradas = pd.read_csv("data\Olist_Funnel_marketing\olist_closed_deals_dataset.csv", parse_dates=['won_date']) 
marketing_qualified_leads = pd.read_csv("data\Olist_Funnel_marketing\olist_marketing_qualified_leads_dataset.csv", parse_dates=['first_contact_date'])

In [19]:
geolocalizacion.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


Hay muchas latitudes y longitudes para un mismo zip-code. Vamos a tomar el promedio de estos (seria como un centroide o similar) y trabajarlo asi

In [28]:
geoloc_filtrado = geolocalizacion.groupby('geolocation_zip_code_prefix').mean().reset_index()

In [15]:
geolocalizacion.geolocation_zip_code_prefix.nunique()

19015

In [32]:
geoloc_filtrado.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,IdGeolocalizacion
0,1001,-23.550190,-46.634024,0
1,1002,-23.548146,-46.634979,1
2,1003,-23.548994,-46.635731,2
3,1004,-23.549799,-46.634757,3
4,1005,-23.549456,-46.636733,4


In [30]:
geoloc_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19015 entries, 0 to 19014
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   geolocation_zip_code_prefix  19015 non-null  int64  
 1   geolocation_lat              19015 non-null  float64
 2   geolocation_lng              19015 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 445.8 KB


In [31]:
# agregamos columna de Id igual a la de index
geoloc_filtrado['IdGeolocalizacion'] = geoloc_filtrado.index

In [54]:
# hacemos el merge con geolocalizacion para traer los nombres de las ciudades y estados

geoloc_filtrado = geoloc_filtrado.join(geolocalizacion, on='geolocation_zip_code_prefix', how='inner', lsuffix='geolocation_zip_code_prefix', rsuffix='geolocation_zip_code_prefix')

In [55]:
geoloc_filtrado.geolocation_zip_code_prefix.nunique()

19015

In [56]:
geoloc_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19015 entries, 0 to 19014
Data columns (total 10 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   geolocation_zip_code_prefix                             19015 non-null  int64  
 1   geolocation_zip_code_prefixgeolocation_zip_code_prefix  19015 non-null  int64  
 2   geolocation_latgeolocation_zip_code_prefix              19015 non-null  float64
 3   geolocation_lnggeolocation_zip_code_prefix              19015 non-null  float64
 4   IdGeolocalizacion                                       19015 non-null  int64  
 5   geolocation_zip_code_prefixgeolocation_zip_code_prefix  19015 non-null  int64  
 6   geolocation_latgeolocation_zip_code_prefix              19015 non-null  float64
 7   geolocation_lnggeolocation_zip_code_prefix              19015 non-null  float64
 8   geolocation_city                    

In [57]:
# eliminamos las columnas que no necesitamos y las duplicadas
geoloc_filtrado.drop(['geolocation_zip_code_prefixgeolocation_zip_code_prefix','geolocation_zip_code_prefixgeolocation_zip_code_prefix',], axis=1,inplace = True)
geoloc_filtrado = geoloc_filtrado.loc[:,~geoloc_filtrado.columns.duplicated()]

In [58]:
geoloc_filtrado

,geolocation_zip_code_prefix,geolocation_latgeolocation_zip_code_prefix,geolocation_lnggeolocation_zip_code_prefix,IdGeolocalizacion,geolocation_city,geolocation_state
0,1001,-23.550190,-46.634024,0,sao paulo,SP
1,1002,-23.548146,-46.634979,1,sao paulo,SP
2,1003,-23.548994,-46.635731,2,sao paulo,SP
3,1004,-23.549799,-46.634757,3,sao paulo,SP
4,1005,-23.549456,-46.636733,4,sao paulo,SP
...,...,...,...,...,...,...
19010,99960,-27.953722,-52.025511,19010,sao paulo,SP
19011,99965,-28.183372,-52.039850,19011,sao paulo,SP
19012,99970,-28.343766,-51.874689,19012,sao paulo,SP
19013,99980,-28.389129,-51.843836,19013,são paulo,SP


In [59]:
# reordenamos y renombramos las columnas
geoloc_filtrado = geoloc_filtrado.reindex(columns=['IdGeolocalizacion', 'geolocation_zip_code_prefix', 
                                                    'geolocation_latgeolocation_zip_code_prefix', 
                                                    'geolocation_lnggeolocation_zip_code_prefix',
                                                    'geolocation_city','geolocation_state'])
geoloc_filtrado.rename(columns={'geolocation_zip_code_prefix': 'zip_code_prefix', 
                                'geolocation_latgeolocation_zip_code_prefix': 'Latitud',
                                'geolocation_lnggeolocation_zip_code_prefix': 'Longitud',
                                'geolocation_city':'Ciudad',
                                'geolocation_state':'Estado'}, inplace=True)

In [70]:
geoloc_filtrado.head()

,IdGeolocalizacion,zip_code_prefix,Latitud,Longitud,Ciudad,Estado
0,0,1001,-23.550190,-46.634024,sao paulo,SP
1,1,1002,-23.548146,-46.634979,sao paulo,SP
2,2,1003,-23.548994,-46.635731,sao paulo,SP
3,3,1004,-23.549799,-46.634757,sao paulo,SP
4,4,1005,-23.549456,-46.636733,sao paulo,SP


In [74]:
geolocalizacion[geolocalizacion.geolocation_city=='sao paulo']

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
...,...,...,...,...,...
207052,8450,-23.547066,-46.391092,sao paulo,SP
207053,8473,-23.572693,-46.394236,sao paulo,SP
207054,8411,-23.555941,-46.413909,sao paulo,SP
207055,8470,-23.589970,-46.394581,sao paulo,SP


In [62]:
import os  
path_etl = "Dataset_etl"
path_aux = "Dataset_aux"
os.makedirs(path_etl, exist_ok=True)  
os.makedirs(path_aux, exist_ok=True)  

# En la carpeta Dataset_etl guardamos los archivos luego del proceso de ETL
geoloc_filtrado.to_csv("{}/geolocalizacion_etl.csv".format(path_etl), index=False)

### ETL a clientes


In [176]:
clientes.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [177]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [178]:
# no tenemos id repetidos
clientes.customer_id.nunique()

99441

In [179]:
# esta bien que hallan repetidos porque los unicos de este item son los clientes, pero estan guardados como compras unicas... 
# con el customer_id que refiere al id de la compra de cada cliente

clientes.customer_unique_id.nunique()

96096

Hacemos una tabla sola de clientes?? en principio les haremos el ETL para agregar el ID de geolocalizacion

Chequeamos que los zip-codes de la tabla clientes estan en la tabla geolocalizacion

In [180]:
geoloc_filtrado.head(2)

,IdGeolocalizacion,zip_code_prefix,Latitud,Longitud,Ciudad,Estado
0,0,1001,-23.550190,-46.634024,sao paulo,SP
1,1,1002,-23.548146,-46.634979,sao paulo,SP


In [186]:
geoloc_unique_code = geoloc_filtrado.zip_code_prefix.unique()
clientes_unique_code = clientes.customer_zip_code_prefix.unique()
zip_codes_dif = [item for item in clientes_unique_code if item not in geoloc_unique_code]
len(zip_codes_dif)

42

Hay 157 codigos postales de clientes que no coinciden con los codigos de la tabla geolocalizacion.
Para cada ciudad en la tabla de geolocalizacion hay mas de un zip-code... lo que vamos a hacer es cambiar los de los clientes por la mediana de los de geolocalizacion

In [184]:
geoloc_unique_code = geolocalizacion.geolocation_zip_code_prefix.unique()
clientes_unique_code = clientes.customer_zip_code_prefix.unique()
zip_codes_missing = []
for item in clientes_unique_code:
    if item not in geoloc_unique_code:
        try:
            city = clientes.loc[clientes['customer_zip_code_prefix'] == item].customer_city.values[0]
            zip_code = int(geolocalizacion[geolocalizacion.geolocation_city==city].geolocation_zip_code_prefix.median())
            clientes['customer_zip_code_prefix'] = clientes['customer_zip_code_prefix'].replace(item, zip_code)
        except:
            zip_codes_missing.append(item)

In [185]:
len(zip_codes_missing)

42

In [189]:
(len(zip_codes_missing)/clientes.shape[0])*100

0.04223609979787009

Todavia tenemos 42 zip-code que no se encuentran en geolocalizacion. Estos corresponden al 0.04% de los datos... probamos con Levenshtein a ver si podemos evaluar nombres mal escritos sino los guardamos en una tabla auxiliar


In [190]:
# ciudades a evaluar
ciudades_eval = []
index = []
for item in zip_codes_missing:
    ciudades_eval.append(clientes.loc[clientes['customer_zip_code_prefix'] == item].customer_city.values[0])
    index.append(clientes.loc[clientes['customer_zip_code_prefix'] == item].index.values[0])

#ciudades de geolocalizacion
ciudades_geoloc = geolocalizacion.geolocation_city.unique()


In [191]:
from hermetrics.levenshtein  import Levenshtein 

lev = Levenshtein ()

resultado_favorable_lev = {}
resultado_desfavorable_lev = {}

for x in ciudades_eval:
    puntuacion = 0
    i=-1
    for y in ciudades_geoloc:
        puntos = lev.similarity(x, y)
        i+=1
        if puntos > puntuacion:
            puntuacion = puntos
            indice = i
    if puntuacion>=0.7:
        resultado_favorable_lev[x] = [ciudades_geoloc[indice], puntuacion]
    else:
        resultado_desfavorable_lev[x] = [ciudades_geoloc[indice],puntuacion]

In [192]:
len(resultado_favorable_lev)

20

In [193]:
len(resultado_desfavorable_lev)

22

In [194]:
resultado_favorable_lev

{'glaura': ['planura', 0.7142857142857143],
 'nossa senhora do remedio': ['nossa senhora dos remedios',
  0.9230769230769231],
 'cipo-guacu': ['ipanguacu', 0.7],
 'sao francisco do humaita': ['sao francisco da praia', 0.7916666666666666],
 'palmeirinha': ['palmeirina', 0.9090909090909091],
 'mampituba': ['marituba', 0.7777777777777778],
 'passagem': ['passabem', 0.875],
 'sao sebastiao da serra': ['sao sebastiao da barra', 0.9090909090909091],
 'doce grande': ['laje grande', 0.7272727272727273],
 'itabi': ['itobi', 0.8],
 'bora': ['borba', 0.8],
 'ibitioca': ['ibitiuva', 0.75],
 'jaguarembe': ['jaguaribe', 0.8],
 'conceicao do formoso': ['conceicao dos ouros', 0.75],
 'poco de pedra': ['pocao de pedras', 0.8666666666666667],
 'jaua': ['maua', 0.75],
 'sao miguel do cambui': ['sao miguel dos campos', 0.8095238095238095],
 'palmital de minas': ['para de minas', 0.7058823529411764],
 'sao sebastiao do paraiba': ['sao sebastiao do paraiso', 0.9166666666666666],
 'siriji': ['siriri', 0.8333

Esta complicado arreglarlos.. asi que en principio guardamos en una tabla auxiliar las filas que no logramos corregir


In [195]:
cientes_aux = clientes.iloc[index]

In [197]:
cientes_aux.shape

(42, 5)

In [198]:
clientes.shape

(99441, 5)

In [199]:
clientes.drop(index = index, axis=0, inplace=True)

In [200]:
clientes.shape

(99399, 5)

Ahora que eliminamos estos valores problema tratemos de unir las tablas geoloc_filtrado con clientes para agregar el id de geolocalizacion

In [201]:
clientes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99399 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99399 non-null  object
 1   customer_unique_id        99399 non-null  object
 2   customer_zip_code_prefix  99399 non-null  int64 
 3   customer_city             99399 non-null  object
 4   customer_state            99399 non-null  object
dtypes: int64(1), object(4)
memory usage: 4.6+ MB


In [204]:
geoloc_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19015 entries, 0 to 19014
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IdGeolocalizacion  19015 non-null  int64  
 1   zip_code_prefix    19015 non-null  int64  
 2   Latitud            19015 non-null  float64
 3   Longitud           19015 non-null  float64
 4   Ciudad             19015 non-null  object 
 5   Estado             19015 non-null  object 
dtypes: float64(2), int64(2), object(2)
memory usage: 1.0+ MB


In [206]:
clientes_join = clientes.join(geoloc_filtrado, on='customer_zip_code_prefix', how='inner', lsuffix='customer_zip_code_prefix', rsuffix='zip_code_prefix')

In [208]:
clientes_join = pd.merge(clientes, geoloc_filtrado, left_on='customer_zip_code_prefix', right_on='zip_code_prefix', how='inner')

In [209]:
clientes_join

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,IdGeolocalizacion,zip_code_prefix,Latitud,Longitud,Ciudad,Estado
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,5358,14409,-20.498489,-47.396929,sao paulo,SP
1,5dca924cc99eea2dc5ba40d11ec5dd0f,2761fee7f378f0a8d7682d8a3fa07ab1,14409,franca,SP,5358,14409,-20.498489,-47.396929,sao paulo,SP
2,661897d4968f1b59bfff74c7eb2eb4fc,d06a495406b79cb8203ea21cc0942f8c,14409,franca,SP,5358,14409,-20.498489,-47.396929,sao paulo,SP
3,702b62324327ccba20f1be3465426437,8b3d988f330c1d1c3332ccd440c147b7,14409,franca,SP,5358,14409,-20.498489,-47.396929,sao paulo,SP
4,bdf997bae7ca819b0415f5174d6b4302,866755e25db620f8d7e81b351a15bb2f,14409,franca,SP,5358,14409,-20.498489,-47.396929,sao paulo,SP
...,...,...,...,...,...,...,...,...,...,...,...
99387,0b7a30ba373aeb55cf28add5b5477956,8c8173e547e020f411aa55b2fceed861,87145,paicandu,PR,17117,87145,-23.501806,-52.150926,sao paulo,SP
99388,ff6b440b2465bfaef05c675efb48cd15,e38a27b979a552e019e297ecc22dc67b,98860,eugenio de castro,RS,18849,98860,-28.527590,-54.145960,sao paulo,SP
99389,3f7029d9b98a47370cd5f3a97adcbccd,5894d202cc869b4ec460fce3c6ed8cb5,5538,sao paulo,SP,2790,5538,-23.588483,-46.737908,sao paulo,SP
99390,d9110683c7a282144e9fc97660026a28,5cbfdb85ec130898108b32c50d619c39,74980,aparecida de goiania,GO,15057,74980,-16.822735,-49.248175,sao paulo,SP


In [213]:
'franca' in geolocalizacion.geolocation_city.unique()

True

In [217]:
geolocalizacion[geolocalizacion.geolocation_city == 'franca']

,IdGeolocalizacion,zip_code_prefix,Latitud,Longitud,Ciudad,Estado


In [218]:
geoloc_filtrado[geoloc_filtrado.zip_code_prefix == 14409].Ciudad.unique()

array(['sao paulo'], dtype=object)